# Analyze SoHappy output files - Significances and timing

In [ ]:
import gammapy
gammapy.__version__

In [ ]:
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes,inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import setup
from setup import col_n, col_s, col_b, dtpop
from pop_plot import col_size, sig_legend, sig_plot_cumulative
import init as init

from utilities import MyLabel, single_legend, stamp

plt.style.use('seaborn-talk') # Make the labels readable
#plt.style.use('seaborn-poster') # Make the labels readable - bug with normal x marker !!!

## Read data - create the total population

In [ ]:
# Open data, get population
file = init.create_csv(debug=False)
(grb, gn0, gs0, gn, gs, gb) = init.get_data(file, debug=False)

# Effective det_level
eff_lvl = init.get_eff_lvl(grb)

import pandas as pd
fullpop = pd.concat([gn0,gs0,gb],axis=0)
len(fullpop)

# <font color="#FF0000">SIGNIFICANCES</font>
Could go in "find best grb" notebook

### GRB with max siginificance obtained after one day

In [ ]:
sigmx_min = 5
tmin = 24*3600
print(" Events above ",sigmx_min,", ",tmin/3600," h after the trigger")

for g,txt in zip([gn, gs, gb, fullpop],["North","South","Both","All"]):
    mask = g.sigmx>sigmx_min
    late_list = g[mask][g[mask].tmx>tmin]
    
    print(" {:5s} {:4d}"":".format(txt,len(late_list)))
    
    for s, t, sigma in zip(late_list.name,late_list.tmx, late_list.sigmx):
        print("{:10s}: t={:10.2f} sig={:10.2f}".format(s,t,sigma),end="")
        if (sigma >= 3): print(" *")
        else: print()
    # late_list.name.values.flatten()

## Max. Significance distributions
The following plot shows that "Both" has better performance (sigma) than N and S separately

In [ ]:
def plot_sigmax(g,ax=None,logx=True,bins=25,tag="",weight=1,xmax=None,**kwargs):
    """
    Plot the max siginificance dsitribution for one population
    """
    
    if (ax==None): fig,ax = plt.subplots()
        
    if (xmax != None): mask = g.sigmx < xmax
    else:              mask= (np.ones(len(g),dtype=bool)) & (g.sigmx!= -1)
        
    if logx:
        mask = (g.sigmx>0) & (mask) 
        x = np.log10(g[mask].sigmx)
        
        ax.set_title("Positive values only")
        ax.set_xlabel("$log_{10}(\sigma_{max})$")   
        ax.axvline(x=np.log10(3),color="tab:orange",ls=":",label="$3\sigma$")
        ax.axvline(x=np.log10(5),color="tab:green",ls=":",label="$5\sigma$") 
    else:
        x = g[mask].sigmx
        ax.set_xlabel("$\sigma_{max}$")
        ax.axvline(x=3.,color="tab:orange",ls=":",label="$3\sigma$")
        ax.axvline(x=5,color="tab:green",ls=":"  ,label="$5\sigma$")   
        
    n, bins,_ = ax.hist(x, bins=bins, label=MyLabel(x,tag,stat="med"),weights=np.ones(len(x))*weight,**kwargs) 
    
    single_legend(ax)
    
    return (ax, bins)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1,ncols=3,figsize=(18,6),sharey=True)
logx = True
nbin=25
ax, bins = plot_sigmax(gn,tag="North",ax=ax1,bins=nbin,logx=logx,color=col_n,alpha=0.5)
plot_sigmax(gn0,tag="North only",ax=ax1,logx=logx,bins=bins,color=col_n,alpha=0.5)

ax, bins = plot_sigmax(gs,tag="South",ax=ax2,bins=nbin,logx=logx,color=col_s,alpha=0.5)
plot_sigmax(gs0,tag="South only",ax=ax,logx=logx,bins=bins,color=col_s,alpha=0.5)

ax, bins = plot_sigmax(gb,tag="Both",
                       logx=logx,color=col_b,ax=ax3,bins=nbin,alpha=0.5)
#plot_sigmax(pd.concat([gn0,gs0],axis=0),tag="N+S",logx=logx,facecolor="none",edgecolor="black",ax=ax3,bins=bins,alpha=1)
import pandas as pd
gtot= pd.concat([gn,gs],axis=0)
plot_sigmax(gtot,tag="N+S",
            logx=logx,facecolor="none",edgecolor="black",ax=ax3,bins=bins,alpha=1,weight=len(gb)/len(gtot))

# ax1.set_yscale("log")
# ax2.set_yscale("log")
# ax3.set_yscale("log")
stamp(file,axis=fig,where="bottom")
plt.tight_layout()

## Highest sigma values

In [ ]:
siglist = [-0.5, 3, 5, 10, 20, 50, 100, 200]
poplist = [gn,gn0,gs,gs0,gb]
taglist = ["North","N only","South","S only","Both"]
duration = 10 # in years

In [ ]:
print(" Detection rate with mean sigmax above sig for ",duration," years")
print(60*"-")

print("{:>10s}".format("Sig min"),end="")
for tag in taglist:
    print("{:>8s}".format(tag),end="")
print("{:>8s}".format("Tot"))
print(60*"-")

for sig in siglist:
    print("{:>10.1f}".format(sig),end="")
    ntot = len(gn0[gn0.sigmx>=sig]) + len(gs0[gs0.sigmx>=sig]) + len(gb[gb.sigmx>=sig])
    for p  in poplist:
        n = len(p[p.sigmx>=sig])
        print("{:>8.2f}".format(duration*n/dtpop),end="")
    print("{:>8.2f}".format(duration*ntot/dtpop))

print(60*"-")

In [ ]:
def high_sigma(gpop,ax=None,inset=True, sigmin= 5, sig2show=100, tag="", show=True, weight=1, **kwargs):
    """
    Check highest value of sigmax
    """

    #n,bins,_ = ax.hist(gpop.sigmx[gpop.sigmx>=sigmin],label=tag, **kwargs)
    var = gpop.sigmx[gpop.sigmx>=sigmin]
    weights = np.ones(len(var))*weight
    n,bins,_ = ax.hist(var,label=MyLabel(var*weights,tag,stat=None), weights = weights,**kwargs)
    #ax.set_yscale("log")
    ax.set_xlabel("$\sigma_{{max}} \geq {}$".format(sigmin))
    ax.set_ylabel("Event rate")
    #ax.set_title("Negative significances excluded")   
    # Inset in log
    if (inset):
        axx = inset_axes(ax, width="75%", height=1.2,loc="upper right")
        nxx,binsxx,_ = axx.hist(np.log10(gpop[gpop.sigmx>0].sigmx),
                           bins=25,color="grey",edgecolor="black", alpha=0.5)
        axx.axvline(np.log10(3),ls=":",color="red",lw="2",label="$3\sigma$")
        axx.axvline(np.log10(5),color="red",lw="2",label="$5\sigma$")
        axx.set_xlabel("Log $\sigma_{max}$")
        axx.legend()
        
        ax.legend(loc="upper left")
    else:
        ax.legend()
       

    # Outliers
    if show:
        outlier=zip(gpop[gpop.sigmx>sig2show].name,gpop[gpop.sigmx>sig2show].sigmx)
        i=1
        for g in outlier:
            y = (0+ (1.5*i + 0.1))*weight
            ax.text(x=g[1],y=y,s=g[0][5:],rotation=45)
            i+=1
    return ax, bins

In [ ]:
fig,ax = plt.subplots(nrows=3, ncols=1,figsize=(10,9))
nbin = 100
weight = 1/(44/duration)
sigmin = 5

ax0, bins = high_sigma(gn, sigmin=sigmin,
                       bins = nbin, color=col_n,ax=ax[0],
                       tag="N", alpha=0.5, weight=weight, inset=False)
high_sigma(gn0,  sigmin=sigmin,
           bins = bins, color=col_n,ax=ax0,tag="N only",show=False, weight=weight,inset=False)
ax0.set_xlabel(None)
ax0.set_title(" Mean $\sigma_{max}$ for "+str(duration)+" years of observation")
ax0.grid("both",ls=":")
# ax0.axhline(weight)
ax0, bins = high_sigma(gs, sigmin=sigmin,
                       bins = nbin, color=col_s,ax=ax[1],tag="S", alpha=0.5, weight=weight,inset=False)
high_sigma(gn0, sigmin=sigmin,
           bins = bins, color=col_s,ax=ax0,tag="S only",show=False, weight=weight,inset=False)
ax0.set_xlabel(None)

ax0, bins = high_sigma(gb, sigmin=sigmin,
                       bins=nbin, color=col_b,ax=ax[2],tag="Both", alpha=0.5,weight=weight, inset=False)
ax0.grid("both",ls=":")
stamp(file,axis=fig)

plt.tight_layout()

## Statistics at high sigma

In [ ]:
nbin=10
cumulative = False
import pandas as pd
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9,6))
first= True
for g, tag, w in zip([fullpop, gb, pd.concat([gn0,gs0],axis=0),pd.concat([gn,gs],axis=0)],
                  ["All", "Both", "Only N+ Only S", "0.5*(N+S)"],
                [1, 1, 1, 0.5]):
    mask = g.sigmx > 0
    var = np.log10(g[mask].sigmx)
    
    if not first: nbin = bins
    n, bins = np.histogram(var, bins = nbin, weights=w*np.ones(len(var)))
    ncum = np.cumsum(n)
    ax.plot(bins[:-1] + 0.5*(bins[1:]-bins[:-1]),(max(ncum)-ncum)*10/dtpop,alpha=1,
            marker="o",label=tag)

#     print(ncum)

    single_legend(ax,bbox_to_anchor=[1.0,1.0])
    ax.grid("both",ls=":")
    ax.axvline(np.log10(50),color="grey",alpha=0.5,ls="--")
    ax.text(np.log10(50)*1.05,0.2,"$50\sigma$",size=12)
    
    ax.axvline(np.log10(20),color="grey",alpha=0.5, ls="--")
    ax.text(np.log10(20)*1.05,0.2,"$20\sigma$",size=12)
            
    ax.axvline(np.log10(5),color="grey",alpha=0.5,ls=":")
    ax.text(np.log10(5)*1.05,0.2,"$5\sigma$",size=12)

    ax.set_xlabel("$Log_{10}$ $\sigma_{max}$")
    ax.set_ylabel("Rate above $\sigma_{max}$ for 10 yr of operation")
    ax.set_yscale("log")
stamp(file, axis=fig, where="bottom")

In [ ]:
siglist = [-0.5, 3, 5, 10, 20, 50, 100, 200]
poplist = [gn, gn0, gs, gs0, gb, fullpop]
taglist = ["North", "N only", "South", "S only", "Both", "All"]

print("{:>8s} ".format("sigmin"),end="")

for tag in taglist:
    print("{:>8s} ".format(tag),end="")
print()
    
for sigmin in siglist:
    print("{:8.1f} ".format(sigmin),end="")
    for g in poplist:
        print("{:8.1f} ".format(len(g[g.sigmx>=sigmin])*10/dtpop),end="")
    print()

# <font color="#FF0000">TIMING</font>

## 3s, 5s and sigmax time distribution
Don't forget the confidence level !!!

In [ ]:
def sig_plot(varlist, masks=None, tags=None, xlabel="", yscale="log", binw=1, tobs=[], **kwargs):
    
    if masks == None: 
        masks = np.ones(len(varlist)).astype(bool)
        
    # Define one hour binning from the min and max
    tmax = int(max([max(v[m]) for v,m in zip(varlist,masks)]))
    bins=range(0, tmax+ 1, binw)
    print(" Max time :",tmax, " bins:",bins)
    
    fig, ax = plt.subplots(nrows=len(varlist), ncols=1,figsize=(12,3.5*len(varlist)),sharex=True)
    
    first = True
    for ax0, tag, var, mask in zip(ax,tags,varlist,masks):
        
        n, bins,_ = ax0.hist(var[mask],bins=bins,label=MyLabel(var[mask],label=tag+" (Burst)",stat="med"),**kwargs)
        if len(tobs) != 0:
            ax0.hist(var[mask]-tobs[mask],bins=bins,label=MyLabel(var[mask]-tobs[mask],label=tag+" (Start)",stat="med"),**kwargs)
        ax0.set_yscale(yscale)
        # ax0.set_title(tag)
        
        ax0.legend()
        ax0.grid(which='both')
        minor_ticks= bins
        ax0.set_xticks(minor_ticks, minor=True)
        
        ax0.grid(which="major",ls=":")
        ax0.grid(which="minor",ls=":",alpha=0.5)
        ax0.set_ylabel("$h^{-1}$") 

        ax0.set_xlabel(xlabel) # Last one
        
        axx = inset_axes(ax0, width="50%", height=1.2,loc="upper center")
        nxx,binsxx,_ = axx.hist(var[mask][var[mask]<1]*60,bins=range(0,60,1),alpha=0.5)
        axx.axvline(107/60,ls=":",label=" Swift + LST delays")
        axx.set_xlabel("$\Delta t$ (min)")
        axx.set_ylabel("$min^{-1}$") 
        axx.legend()
        
    stamp(file,axis=fig)    
    plt.tight_layout()

In [ ]:
# Don't forget the confidence level !!!!
# varlist  = [fullpop.t3s/3600, fullpop.t5s/3600, fullpop.tmx/3600] # Go to hours
# masklist = [(fullpop.d3s>eff_lvl), (fullpop.d5s>eff_lvl), (fullpop.tmx>=0) & (fullpop.sigmx>=5)]
# taglist = ["$3 \sigma$","$5\sigma$","$\sigma_{max} \geq 5$"]

varlist  = [fullpop.t5s/3600, fullpop.tmx/3600] # Go to hours
masklist = [(fullpop.d5s>eff_lvl), (fullpop.tmx>=0) & (fullpop.sigmx>=5)]
taglist = ["$5\sigma$","$\sigma_{max} \geq 5$"]
sig_plot(varlist,masks=masklist,tags=taglist, xlabel="Mean detection time (h)",binw=1, tobs = fullpop.t1/3600,yscale="log",alpha=0.5)


In [ ]:
pop =fullpop
    
varlist  = [np.log10(pop.t5s), np.log10(pop.tmx)] # Go to hours
masklist = [(pop.d5s>eff_lvl), (pop.tmx>=0) & (fullpop.sigmx>=5)]
taglist = ["$5\sigma$ 90%CL","$\sigma_{max} \geq 5$"]

fig, ax = plt.subplots(ncols=len(varlist), nrows=1,figsize=(7*len(varlist), 5),sharey=True)

first = True
for ax0, tag, var, mask in zip(ax,taglist,varlist,masklist):
    sig_plot_cumulative(var,mask=mask,tag=tag, 
                        ax=ax0, xlabel="Log detection time (s)")
    if first:
        ax0.set_ylabel("Fraction") 
        first = False
        
single_legend(ax0,bbox_to_anchor=[1,1])
stamp(file,axis=fig,where="left")
plt.tight_layout()

In [ ]:
pop =fullpop
varlist  = [np.log10(gn.t5s), np.log10(gs.t5s)] # Go to hours
masklist = [(gn.d5s>eff_lvl), (gs.d5s>eff_lvl)]
taglist = ["$5\sigma$ North","$5\sigma$ South"]
fig, ax = plt.subplots(ncols=len(varlist), nrows=1,figsize=(7*len(varlist), 5),sharey=True)

first = True
for ax0, tag, var, mask in zip(ax,taglist,varlist,masklist):
    sig_plot_cumulative(var,mask=mask,tag=tag, 
                        ax=ax0, xlabel="Log detection time (s)")
    if first:
        ax0.set_ylabel("Fraction") 
        first = False
        
single_legend(ax0,bbox_to_anchor=[1,1])
stamp(file,axis=fig,where="left")
plt.tight_layout()

In [ ]:
## Time versus error on time
def sig2_plot(varlist1, varlist2, masklist, xlabel="", ylabel="", xscale="log", yscale="log", nbin=100, **kwargs):
    
    fig, ax = plt.subplots(nrows=1, ncols=3,figsize=(18,6),sharey=True)
    taglist = ["$3 \sigma$","$5\sigma$","$\sigma_{max}$"]
    
    first=True
    for ax0, tag, var1, var2, mask in zip(ax,taglist,varlist1,varlist2, masklist):
        ax0.scatter(var1[mask], var2[mask],**kwargs)
        ax0.set_xscale(xscale)
        ax0.set_yscale(yscale)
        ax0.set_xlabel(xlabel)
        if first: 
            ax0.set_ylabel(ylabel) 
        else: 
            ax0.set_ylabel(None)
            first=False        
        ax0.set_title(tag)
        #ax0.legend()
        ax0.grid(ls=":")
        
    stamp(file,axis=fig)    
    plt.tight_layout()

In [ ]:
varlist1 = [fullpop.t3s/3600, fullpop.t5s/3600, fullpop.tmx/3600]
varlist2 = [fullpop.et3s/3600, fullpop.et5s/3600, fullpop.etmx/3600]
masklist = masklist = [(fullpop.d3s>eff_lvl), (fullpop.d5s>eff_lvl), (fullpop.tmx>=0) & (fullpop.sigmx>=5)]

sig2_plot(varlist1,varlist2,masklist,xlabel="Detection mean time",ylabel="Error on time",marker="+")
#sig2_plot(varlist1,varlist2,masklist,xlabel="Detection mean time",ylabel="Error on time",xscale="linear",yscale="linear",marker="+")

# <font color="#FF0000">SIGNIFICANCE versus TIMING</font>

## Raw plot

In [ ]:
def plot_t_sigma(g,tag=""):
    fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(12,6),sharex=True)

    mask = g.sigmx>0
    colors, sizes = col_size(g[mask].sigmx)
    ax.scatter(np.log10(g[mask].tmx),np.log10(g[mask].sigmx),marker="o",color=colors,alpha=0.7,s=sizes)
    ax.scatter(np.log10(g[mask].tmx),np.log10(g[mask].sigmx),marker="x",alpha=0.5,s=5,color="grey")

    ax.set_xlabel("$log_{10}(\Delta t_{max})$")
    ax.set_ylabel("$log_{10}(\sigma_{max})$")
    o1 = ax.axvline(x=np.log10(3600),ls=":",color="green",label="1h hour")
    o2 = ax.axvline(x=np.log10(12*3600),ls=":",color="blue",label="12h hour")
    o3 = ax.axvline(x=np.log10(24*3600),ls=":",color="red",label="One day")
    o4 = ax.axhline(np.log10(3),ls=":",color="black",lw="2",label="$3\sigma$")
    o5 = ax.axhline(np.log10(5),color="black",lw="2",label="$5\sigma$")
    
    ax.legend(loc="lower left")
    
    patches = sig_legend(ax)
    fig.legend(title="$\sigma_{max}$",handles=patches, bbox_to_anchor=(1.1, 0.98))
    stamp(file,axis=fig,where="top")    
    return
plot_t_sigma(fullpop,tag="All")
plt.tight_layout()

In [ ]:
# Plot sigmax and errors
def sig_vs_time(pop,mask,ax=None,xscale="log",yscale="log",title=""):
    ax = plt.gca() if ax is None else ax
    ax.errorbar(pop[mask].tmx/3600,
                pop[mask].sigmx,
                xerr=pop[mask].etmx/3600,
                yerr=pop[mask].esigmx,
                ls="", marker="o",ecolor="tab:blue",color="red",alpha=0.5,
                label=MyLabel(pop[mask].sigmx,stat=None))
    # ax.axhline(y=3,ls=":",label="$3\sigma$",color="lightgrey")
    # ax.axhline(y=5,ls=":",label="$5\sigma$",color="lightgrey")
    ax.set_xscale(xscale)
    ax.set_yscale(yscale)
    ax.set_xlabel("Mean Time of max detection (h)")
    ax.set_ylabel("Mean max sigificance")
    ax.set_title(title)
    ax.legend()

In [ ]:
fig,(ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(15,8))
sig_vs_time(fullpop,(fullpop.sigmx>=20),
            yscale="log",ax=ax1, title="$\sigma_{max} > 20$")
sig_vs_time(fullpop,(fullpop.sigmx>=5) & (fullpop.sigmx<20),
            xscale="linear",yscale="linear",ax=ax2,title="$5 < \sigma_{max} < 20$")
stamp(file,axis=fig)

## Delay correlation for GRB seen on both sites
<font color="#FF0000">Warning</font>: works if accelaration was False, i.e. a results was obatined fro all sites (Abortion due to acceleration breaks the N/S symmetry)

In [ ]:
# Find N and S detection for all both detected GRB
# Get name of GRB detected on both sites and retrieve the independent N and S detections
t1_s = []
t1_n = []
sigmx = []
sigmin = 5

for name in gb.name:
    gsb = gs[gs.name==name] # This is the row
    gnb = gn[gn.name==name]
#     print(" ****", name,":")
#     print("   t1 N = ",gnb.t1.values)
#     print("   t1 S = ",gsb.t1.values)
    sig = gb[gb.name == name].sigmx.values[0]
    if sig>= sigmin:
        sigmx.append(gb[gb.name == name].sigmx.values[0]) 
        t1_s.append(gsb.t1.values[0]/3600)
        t1_n.append(gnb.t1.values[0]/3600)    


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,7))
color, size = col_size(sigmx)
sc = ax.scatter(t1_n, t1_s,  alpha=0.5,c= color, s=size, label = MyLabel(t1_n,"$\sigma_{max}$"))
# for tn, ts, sig in zip(t1_n,t1_s, sigmx):
#     ax.text(tn,ts,str(sig))
ax.set_title("Seen on both sites - $\sigma_{max} \geq "+str(sigmin)+"$")
ax.set_xlabel("Mean detection time (h) - North")
ax.set_ylabel("Mean detection time (h) - South")
xmax = 26
ymax = 26
ax.set_xlim(xmax=26)
ax.set_ylim(ymax=26)

# ZOOM
xmin_z = 0
xmax_z = 2
ymin_z = 0
ymax_z = 5
ax.axvspan(xmin_z-1,xmax_z,ymin=ymin_z-1,ymax=(ymax_z+1)/ymax,color="grey",ls=":",alpha=0.5,lw=2,ec="black",fill=False)

axx = inset_axes(ax, width="30%", height=3,loc="upper right")
sc = axx.scatter(t1_n, t1_s,  alpha=0.5,c= color, s=size, label = MyLabel(t1_n,"$\sigma_{max}$"))
axx.set_xlim(xmax=xmax_z)
axx.set_ylim(ymax=ymax_z)

stamp(file, axis=fig,where="bottom")
patches = sig_legend(ax)
fig.legend(title="$\sigma_{max}$",handles=patches,bbox_to_anchor=(1.02, 0.88))